In [9]:
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from itertools import combinations
from scipy.linalg import LinAlgError
from scipy.spatial.distance import cdist
import pandas as pd
from modelsgelu import *
import numpy as np
import optuna
import random

In [2]:
def seed_everything(seed):
    torch.manual_seed(seed)       # Current CPU
    torch.cuda.manual_seed(seed)  # Current GPU
    np.random.seed(seed)          # Numpy module
    random.seed(seed)             # Python random module
    torch.backends.cudnn.benchmark = False    # Close optimization
    torch.backends.cudnn.deterministic = True # Close optimization
    torch.cuda.manual_seed_all(seed) # All GPU (Optional)
seed_everything(1)
load_study = optuna.study.load_study("selus", "sqlite:///optuna.db")
best_params = load_study.best_params
model_dim = best_params["model_dim"]
# batch_size = best_params["batch_size"]
batch_size = 128
lr = best_params["lr"]
skin_hidden_size = best_params["skin_hidden_size"]
stringer_hidden_size = best_params["stringer_hidden_size"]
otherfeatures_hidden_size = best_params["otherfeatures_hidden_size"]
net = parallel_net(model_dim,skin_hidden_size,stringer_hidden_size,otherfeatures_hidden_size)
bucking_loads = np.zeros(1)
preds = np.zeros(1)
model_dict = torch.load("./checkpoint5.pt")
net.load_state_dict(model_dict)
net.eval()

C:\Users\user\AppData\Local\Temp\ipykernel_62448\1812033132.py:10: FutureWarning: load_study(): Please give all values as keyword arguments. See https://github.com/optuna/optuna/issues/3324 for details.
  load_study = optuna.study.load_study("selus", "sqlite:///optuna.db")


parallel_net(
  (skin_embedding_tabel): Embedding(5, 4096)
  (stringer_embedding_tabel): Embedding(5, 4096)
  (Bi_lstm_skin): LSTM(4096, 2048, batch_first=True, bidirectional=True)
  (Bi_lstm_stringer): LSTM(4096, 1024, batch_first=True, bidirectional=True)
  (attention_skin): SelfAttention(
    (linear_q): Linear(in_features=4096, out_features=4096, bias=False)
    (linear_k): Linear(in_features=4096, out_features=4096, bias=False)
    (linear_v): Linear(in_features=4096, out_features=4096, bias=False)
  )
  (attention_stringer): SelfAttention(
    (linear_q): Linear(in_features=2048, out_features=2048, bias=False)
    (linear_k): Linear(in_features=2048, out_features=2048, bias=False)
    (linear_v): Linear(in_features=2048, out_features=2048, bias=False)
  )
  (feedforward_skin): FeedForward(
    (w_1): Linear(in_features=4096, out_features=2048, bias=True)
    (w_2): Linear(in_features=2048, out_features=4096, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (feedforwa

In [13]:
Material_lab = pd.read_csv("./ml.csv",header=None)
Material_lab

,0,1,2,3,4,5,6
0,134.700,7.700,4.200,4.200,2.500,0.369,1590.0
1,145.000,9.500,5.000,5.000,3.500,0.300,1450.0
2,101.700,6.000,2.400,2.400,2.400,0.300,1700.0
3,121.000,5.820,4.000,4.000,4.000,0.300,1580.0
4,126.100,11.200,5.460,5.460,5.460,0.300,1560.0
5,25.000,25.000,3.000,3.000,3.000,0.300,2200.0
6,131.000,8.000,4.500,4.500,3.500,0.290,1544.0
7,77.460,77.460,4.150,3.625,3.626,0.030,1655.0
8,155.000,12.100,4.400,4.400,3.200,0.248,1600.0
9,127.700,9.670,6.500,6.500,3.600,0.300,1520.7


In [15]:
lb = np.array([4]*28+[175]*2+[35]*2)

In [16]:
ub = np.array([0]*28+[125]*2+[1]*2)

In [39]:
nvar = len(lb)

In [44]:
pop = np.random.randint(ub,lb+1 ,(100,nvar))
pop

array([[  4,   2,   1, ..., 141,  11,  12],
       [  3,   0,   3, ..., 150,  33,  16],
       [  3,   4,   0, ..., 138,  23,  14],
       ...,
       [  2,   0,   3, ..., 125,  32,   1],
       [  0,   4,   1, ..., 141,   5,  27],
       [  0,   4,   4, ..., 147,   3,  10]])

In [45]:
skin_seq = pop[:,:16]
stringer_seq = pop[:,16:28]
other_feas = np.zeros_like(a=0,shape=(100, 14),dtype=np.float32)


In [46]:
skin_seq_length = np.zeros(100)
strigner_seq_length = np.zeros(100)
rou_skin = np.zeros(100)
rou_stringer = np.zeros(100)

In [47]:
for i in range(100): # type: ignore
    skin_seq_ = skin_seq[i,:]
    decode_skin = np.array([x for x in skin_seq_ if x!=0])
    skin_seq_length[i] = len(decode_skin)
    decode_skin = np.pad(decode_skin, (0,16-len(decode_skin)), "constant", constant_values=0)
    skin_seq[i] = decode_skin
    stringer_seq_ = stringer_seq[i,:]
    decode_stringer = np.array([x for x in stringer_seq_ if x!=0])
    strigner_seq_length[i] = len(decode_stringer)
    decode_stringer = np.pad(decode_stringer, (0,12-len(decode_stringer)), "constant", constant_values=0)
    stringer_seq[i] = decode_stringer
    skin_material = pop[:,28:][i,2]
    stringer_material = pop[:,28:][i,3]
    rou_skin[i] = Material_lab.iloc[skin_material-1][6]
    rou_stringer[i] = Material_lab.iloc[stringer_material-1][6]
    thickness = np.array([pop[:,28:][i,0]/1000 ,pop[:,28:][i,1]/1000])
    skin_material = np.array(Material_lab.iloc[skin_material-1][0:6])
    stringer_material = np.array(Material_lab.iloc[stringer_material-1][0:6])
    other_fea = np.concatenate([skin_material,stringer_material,thickness])  
    other_feas[i] = other_fea


In [50]:
skin_seq[5]

array([3, 1, 3, 1, 3, 3, 2, 4, 1, 2, 3, 1, 4, 4, 2, 0])

In [61]:
def determing_seq(seq):
    pops = seq.shape[0]
    index = np.zeros(pops,dtype=np.int32)
    for i in range(pops):
        seq_ = seq[i]
        decode_seq = np.array([x for x in seq_ if x!=0])
        seq_len = len(decode_seq)
        index[i] = 0
        
        if sum(decode_seq==2) != sum(decode_seq==4) or sum(decode_seq==1)<0.1*seq_len or sum(decode_seq==2)<0.1*seq_len or sum(decode_seq==3)<0.1*seq_len or sum(decode_seq==4)<0.1*seq_len or decode_seq[-3] == decode_seq[-2] or decode_seq[-3] == decode_seq[-1]:
            index[i] = 1

        # for j in range(seq_len-1):
        #     if abs(decode_seq[j]-decode_seq[j+1]) == 2:
        #         index[i] = 1
            
        # for k in range(seq_len-4):
        #      if decode_seq[k] == decode_seq[k+1] or decode_seq[k] == decode_seq[k+2] or decode_seq[k] == decode_seq[k+3] or decode_seq[k] == decode_seq[k+4]:
        #          index[i] = 1
            
    return index


In [72]:
determing_seq(skin_seq)
determing_seq(stringer_seq)
index = np.logical_or(determing_seq(stringer_seq),determing_seq(skin_seq))

In [70]:
other_feas[0,-2]
True or False

True

In [82]:
mass = (0.479+0.000057692*rou_stringer*strigner_seq_length*2*other_feas[:,-1]+0.000422451*rou_skin*skin_seq_length*2*other_feas[:,-2]).reshape(-1,1)
mass[index]+=10000
mass
len(mass)



100

In [15]:
skin_seq = torch.tensor(skin_seq, dtype=torch.int32)
stringer_seq = torch.tensor(stringer_seq, dtype=torch.int32)
other_feas = torch.tensor(other_feas,dtype=torch.float32)

In [16]:
preds = -1*net(skin_seq, stringer_seq, other_feas).detach().numpy()

In [17]:
objs = np.concatenate([mass, preds])

In [16]:
mass = (0.479+0.000057692*1600*10*2*0.14*4+0.000422451*1600*12*2*0.14)
mass

3.7839372160000004

In [6]:
0.000422451*1600*12*2*0.14
0.000057692*1600*10*2*0.14*4

1.0338406400000002